In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Bu ödevde kadın erkek seslerini lojistik regresyon kullanarak cost değerini en aza indirmeye çalışacağız.

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
data = pd.read_csv("../input/voicegender/voice.csv") # datamız otomatik olarak import edildi.

* **Datamızın içerisinde olan "label" adlı feature değerleri string tipinde olduğundan bunu sayısal değere dönüştürüyoruz.Dönüştürdükden sonra info komutu ile datayı tekrar ekrana yazdırarak kontrol ettik.**

In [ ]:
data.label=[1 if each =="male" else 0 for each in data.label]
print(data.info())

* Gördüğünüz gibi label adlı feature int64 tipinde oldu.

* Kadın erkek datamızın bulunduğu feature değerlerini y değişkenine, bu feature'ın olmadığı diğer dataları da x_data adlı değişkenimize atadık.

* reshape(-1,1) komutunu kullanarak sklearn kütüphanesi ile datamızı(matris) uyumlu hale getiriyoruz.
* axis = 1 datamızın stunları anlamına geliyor.

In [ ]:
y=data.label.values.reshape(-1,1)
x_data=data.drop(["label"],axis=1)

* x_datamızın içerisinde bulunan featurelar aynı aralıkta olmadığından bu değerler işlendiğinde herhangi bir feature değerleri küçük olduğunda bir başka feature değeri o değerleri götürebilir dolayısıyla bunları eşit bir şekilde dağıtarak 0 ile 1 arasına her bir feature ı scale ediyoruz.

In [ ]:
x=(x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data)).values

* Verilerimizin %20 sini eğittiğimiz %80 lik bölümüyle test etmek için ayırma işlemini yaptık.

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

* Verileri daha rahat görüntülemek için bunların transpozunu aldık.

In [ ]:
x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

* weight ve bias değişkenlerimize default değerleri attık.
* Sigmoid fonksiyonu yazıldı.

In [ ]:
def initialize_weights_and_bias(dimention):
    w=np.full((dimention,1),0.01)
    b=0.0
    return w,b

#w,b=initialize_weights_and_bias(30)
    
def sigmoid(z):
    y_head=1/(1+np.exp(-z))
    return y_head

* Her bir elemanına 0.01 değeri atılan weight matrisimizi x train ile çarpıp bias değerimizle topladık.Daha sonra bu z değerimizi sigmoid fonksiyonuna sokarak her bir değerimizi 0 ile 1 değeri arasına scale ettik.
* Loss fonksiyonunu bulduk ve bunların toplamının ortalamasından cost değerimizi bulduk.
* weight ve bias değişkenlerin türevlerini aldık.
* cost ve gradients değerlerini döndürdük.


In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    z=np.dot(w.T,x_train)+b
    y_head=sigmoid(z)
    loss=-y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost=(np.sum(loss))/x_train.shape[1]
    
    derivative_weight=(np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias=np.sum(y_head-y_train)/x_train.shape[1]
    gradients={"derivative_weight":derivative_weight,"derivative_bias":derivative_bias}
    
    return cost,gradients

* for döngüsü içerisinde verdiğimiz iterasyon sayısı kadar weight ve bias değerlerimizi güncelledik.

In [ ]:
def update (w,b,x_train,y_train,learning_rate,number_of_iteration):
    cost_list=[]
    cost_list2=[]
    index=[]
    
    for i in range(number_of_iteration):
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        
        w=w-learning_rate*gradients["derivative_weight"]
        b=b-learning_rate*gradients["derivative_bias"]
        if i%10==0:
            cost_list2.append(cost)
            index.append(i)
            print("Cost after iteration: %i: %f"%(i,cost))
            
    parameters={"weight":w,"bias":b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation="vertical")
    plt.xlabel("Number of Iteration")
    plt.ylabel("Cost")
    plt.show()
    return parameters,gradients,cost_list

* Değerlerimizi pretict ediyoruz optimize hale getiriyoruz.

In [ ]:
def predict(w,b,x_test):
    z=sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(z.shape[1]):
        if z[0,i]<=0.5:
            Y_prediction[0,i]=0
        else:
            Y_prediction[0,i]=1
    return Y_prediction

* Verilerimizi update fonksiyonuna gönderdik güncellenip en optimize hale geldikten sonra accuracy mizi ekranda görüntüledik.

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learning_rate,num_iterations):
    
    dimension = x_train.shape[0]
    
    w,b=initialize_weights_and_bias(dimension)
    
    parameters,gradients,cost_list=update(w,b,x_train,y_train,learning_rate,num_iterations)
    
    y_prediction_test=predict(parameters["weight"],parameters["bias"],x_test)
    
    print("test accuracy: {}".format(100-np.mean(np.abs(y_prediction_test-y_test))*100))
    
logistic_regression(x_train,y_train,x_test,y_test,learning_rate=3,num_iterations=1000)

* Görüldüğü gibi cost değerimiz 10 iterasyona kadar artmış sonrasında düşmüştür.